In [1]:
'''Created by Jake Fredrich'''
'''Foursquare Exploration - Applied Data Science'''
'''Date Created: 11/10/2019'''

'Date Created: 11/10/2019'

# Foursquare Exploration

In [1]:
'''IMPORTS'''
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library to handle requests
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# transforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print("Folium installed")
print('Libraries imported.')

Solving environment: ...working... 
  - anaconda::ca-certificates-2019.1.23-0, anaconda::openssl-1.1.1b-he774522_1
  - anaconda::ca-certificates-2019.1.23-0, defaults::openssl-1.1.1b-he774522_1
  - anaconda::openssl-1.1.1b-he774522_1, defaults::ca-certificates-2019.1.23-0
  - defaults::ca-certificates-2019.1.23-0, defaults::openssl-1.1.1b-he774522_1done

## Package Plan ##

  environment location: C:\Users\JAKEFREDRICH\Anaconda3\envs\BaseClone

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/n

In [2]:
'''SET UP FOURSQUARE CREDENTIALS'''
CLIENT_ID = "E512LWZLNEUIVHYK45TVYJXP0WFGT2YB2JWHRO4BMI51BZZW"
CLIENT_SECRET = "XM1RAWJUYHWTT5P1SPKGJNWRYCR2J2EMBZXAJH0D3JCVQRAW"
VERSION = '20180604'
LIMIT = 30
print('Your Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your Credentials:
CLIENT_ID: E512LWZLNEUIVHYK45TVYJXP0WFGT2YB2JWHRO4BMI51BZZW
CLIENT_SECRET:XM1RAWJUYHWTT5P1SPKGJNWRYCR2J2EMBZXAJH0D3JCVQRAW


In [4]:
# Convert Conrad Hotel Address to latitude and longitude coordinates using geocoder
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [7]:
# Search for a specific venue category.  Search for italian food within 500 metres from Conrad Hotel
search_query = 'Italian'
radius = 500
print(search_query + '.... OK!')

Italian.... OK!


In [11]:
# Save URL for call
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=E512LWZLNEUIVHYK45TVYJXP0WFGT2YB2JWHRO4BMI51BZZW&client_secret=XM1RAWJUYHWTT5P1SPKGJNWRYCR2J2EMBZXAJH0D3JCVQRAW&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

In [12]:
'''STORE RESULTS from FOURSQUARE'''
# Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc972f9bcbf7a002cf18e67'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '294544',
     'url': 'https://www.seamless.com/menu/harrys-italian-pizz

In [15]:
# Get the relevant part of JSON and transform it into a pandas dataframe

# assign relevant part of JSON to venues
venues = results['response']['venues']

# transform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<class 'dict'>


,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-p...,False,4fa862b3e4b0ebff2f749f06,225 Murray St,...,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1573483257
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,...,NaN,469,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71448400000000...",40.714484,-74.009806,10007,NY,Conca Cucina Italian Restaurant,v-1573483257
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,...,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,Ecco,v-1573483257


In [62]:
'''FORMAT DATAFRAME - RELEVANT INFO'''
# Define information of interest and filter dataframe

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories', 'id'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
       
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
#print(dataframe_filtered['categories'].head())

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,id,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,4fa862b3e4b0ebff2f749f06,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,4f3232e219836c91c7bfde94,63 W Broadway,US,New York,United States,NaN,469,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71448400000000...",40.714484,-74.009806,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,3fd66200f964a520f4e41ee3,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [63]:
'''VISUALIZATION'''
dataframe_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
2                               Ecco
Name: name, dtype: object

In [64]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color='red',
    fill_opacity= 0.6
    ).add_to(venues_map)
venues_map

#add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color = 'blue',
        fill_opacity=0.6
    ).add_to(venues_map)
#display map
venues_map

In [121]:
'''EXPLORE VENUE'''
venue_id  = dataframe_filtered[dataframe_filtered['name'] == "Harry's Italian Pizza Bar"]
venue_id = venue_id['id'].iloc[0,:].values[0] # ID of Harry's Italian Pizza Bar
print(venue_id)

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

4fa862b3e4b0ebff2f749f06


'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=E512LWZLNEUIVHYK45TVYJXP0WFGT2YB2JWHRO4BMI51BZZW&client_secret=XM1RAWJUYHWTT5P1SPKGJNWRYCR2J2EMBZXAJH0D3JCVQRAW&v=20180604'

In [122]:
# Get Request
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restauran

In [123]:
'''GET VENUE'S RATING'''
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


In [126]:
# Check second restaurant ratings
venue_id  = dataframe_filtered[dataframe_filtered['name'] == "Conca Cucina Italian Restaurant"]
venue_id = venue_id['id'].iloc[0,:].values[0] # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# Get Request
result = requests.get(url).json()
#print(result['response']['venue'].keys())
result['response']['venue']

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [127]:
# Check third restaurant ratings
venue_id  = dataframe_filtered[dataframe_filtered['name'] == "Ecco"]
venue_id = venue_id['id'].iloc[0,:].values[0] # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# Get Request
result = requests.get(url).json()
#print(result['response']['venue'].keys())
result['response']['venue']

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.9


In [132]:
# Print the number of tips
print("tips " + str(result['response']['venue']['tips']['count']))

## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

tips 17


{'meta': {'code': 200, 'requestId': '5dc98d19787dba003861e9d9'},
 'response': {'tips': {'count': 17,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 3,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'El-Tawil',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}},
     'authorInteractionType': 'liked'}]}}}

In [140]:
# Get tips and list of associated features
tips = results['response']['tips']['items']

tips = results['response']['tips']['items'][0]
print(tips.keys())

# Format Columns
pd.set_option('display.max_colwidth',-1)
tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName',\
                    'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",3,0,5ab1cb46c9a517174651d3fe,Nick,El-Tawil,male,484542633


In [141]:
# Search Foursquare User
user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [144]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])
user_data['tips']


First Name: Nick
Last Name: El-Tawil
Home City: New York, NY


{'count': 237}

In [145]:
# Define User tips
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"The best! I’m especially fond of the salmon burger, but I’ve had half of the menu and never been disappointed. There’s a reason this place is well known even outside of the Village!",1,0,5aec594b1f7440002c138612
1,I used to down a pint of chocolate like it was nothing back when I was bulking. Highly recommended!,1,0,5accc9f66fa81f196724807b
2,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
3,"I’m a fan. In fact, I’m such a big fan, I want Taim to hire me to be their spokesman. Kind of like the Arabic Jared Fogle - but without the kid stuff.",1,0,5accbf033abcaf09a24612a0
4,The linguine with clams is on point 👌,1,0,5accbe3a911fc423730f3ed3
5,"Great for a quick, cheap lunch! Shorter lines than Chipotle too👌",1,0,5acbecb86fa81f1967e019b0
6,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",1,0,5acbec70a0215b732e264fe8
7,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
8,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
9,Coffee game on point,1,0,5acbbb1501235808d5d6525e


In [146]:
#tip with greatest agree counts
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73373338282062, 'lng': -74.0062998849649, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73373338282062, 'lng': -74.0062998849649}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


In [148]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

""


In [149]:
user_data # find prefix and suffix of photo.  Add them to URL below
# Image prep
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

In [150]:
# Explore areas surrounding Ecco
latitude = 40.715337
longitude = -74.008848
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

import requests
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))
items = results['response']['groups'][0]['items']
items[0]

dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Korin,Furniture / Home Store,57 Warren St,US,New York,United States,Church St,73,"[57 Warren St (Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}]",40.714824,-74.009404,Tribeca,10007,NY,4af5d65ff964a52091fd21e3
1,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,US,New York,United States,btwn Greenwich St & W Broadway,202,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",40.714788,-74.011132,NaN,10007,NY,54148bc6498ea7bb8c05b70a
2,Takahachi Bakery,Bakery,25 Murray St,US,New York,United States,at Church St,187,"[25 Murray St (at Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}]",40.713653,-74.008804,NaN,10007,NY,4c154c9a77cea593c401d260
3,Chambers Street Wines,Wine Shop,148 Chambers St,US,New York,United States,btwn West Broadway & Hudson St,88,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}]",40.715773,-74.009718,NaN,10007,NY,4adcf23cf964a520cc6221e3
4,Takahachi,Sushi Restaurant,145 Duane St,US,New York,United States,btwn W Broadway & Church St,146,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}]",40.716526,-74.008101,NaN,10013,NY,4a8f2f39f964a520471420e3
5,Los Tacos No. 1,Taco Place,136 Church St,US,New York,United States,NaN,130,"[136 Church St, New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71417, 'lng': -74.00866}]",40.714170,-74.008660,NaN,10007,NY,5d5f24ec09484500079aee00
6,Heyday,Spa,92 Reade St,US,New York,United States,NaN,86,"[92 Reade St, New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.715598486687675, 'lng': -74.00788227511288}]",40.715598,-74.007882,NaN,10013,NY,57ad129c498e05b086594d72
7,Philip Williams Posters,Antique Shop,122 Chambers St,US,New York,United States,NaN,8,"[122 Chambers St, New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71528423132827, 'lng': -74.00878093952018}]",40.715284,-74.008781,NaN,10007,NY,4b747291f964a52042dd2de3
8,Equinox Tribeca,Gym,54 Murray Street,US,New York,United States,at W Broadway,154,"[54 Murray Street (at W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",40.714099,-74.009686,NaN,10007,NY,4a6e331af964a52031d41fe3
9,Nish Nūsh,Falafel Restaurant,88 Reade St,US,New York,United States,at Church St,97,"[88 Reade St (at Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71553710116416, 'lng': -74.00772452925565}]",40.715537,-74.007725,NaN,10013,NY,50ba9119e4b071a4bae6dc10


In [151]:
# CREATE MAP
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [152]:
# Explore Trending Venues
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc99267db1d8100396619d3'},
 'response': {'venues': []}}

In [153]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [157]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [159]:
# create another map and show trending venues
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'